In [5]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from skimage import io, measure
import geopandas as gpd
import rasterio
from rasterio.features import rasterize


In [6]:


# Set the working directory
import os
os.chdir('C:/Users/joao.kreitlon/Documents/IME/2023.1/PDI/VE4/data')

# Read the image
img = io.imread('WV3_VE3.tif')

# Rename the image channels
img_channel_names = ["Coastal","Blue","Green","Yellow","Red","Red_Edge","NIR1","NIR2"]
img = pd.DataFrame(img.reshape(-1, len(img_channel_names)), columns=img_channel_names)

# Load training and test shapefiles
shp_treino = gpd.read_file('classes.shp')
shp_teste = gpd.read_file('test.shp')

# Rasterize the shapes
raster_base = img['Coastal'].values.reshape(img.shape[:-1])
treino = measure.rasterize(shp_treino['geometry'], out_shape=raster_base.shape, fill=np.nan, transform=shp_treino.crs)

# Classify with Mahalanobis distance
def mahalanobis_distance(x, mean, covariance):
    x_minus_mean = x - mean
    inv_covariance = np.linalg.inv(covariance)
    mahalanobis_dist = np.sqrt(np.dot(np.dot(x_minus_mean, inv_covariance), x_minus_mean.T))
    return mahalanobis_dist


AttributeError: module 'skimage.measure' has no attribute 'rasterize'

In [ ]:

# Define class labels
class_labels = ['VEGETACAO ALTA', 'AGUA', 'VEGETACAO RALA', 'TERRA', 'EDIFICACOES', 'CALCADA']

# Initialize empty arrays for mean and covariance matrices
mean_matrices = []
covariance_matrices = []

# Calculate mean and covariance matrices for each class
for class_label in class_labels:
    class_pixels = img.loc[treino['CLASSE'] == class_label, img_channel_names].values
    class_mean = np.mean(class_pixels, axis=0)
    class_covariance = np.cov(class_pixels.T)
    mean_matrices.append(class_mean)
    covariance_matrices.append(class_covariance)

# Calculate Mahalanobis distance for each pixel
mahalanobis_distances = np.empty((img.shape[0], len(class_labels)))
for i, pixel in enumerate(img[img_channel_names].values):
    for j, (class_mean, class_covariance) in enumerate(zip(mean_matrices, covariance_matrices)):
        mahalanobis_distances[i, j] = mahalanobis_distance(pixel, class_mean, class_covariance)

# Assign class label based on the minimum distance
pixel_classes = np.argmin(mahalanobis_distances, axis=1) + 1

# Reshape pixel_classes to match image shape
img_class_mahalanobis = pixel_classes.reshape(raster_base.shape)

# Plot the classified image using scikit-image
io.imshow(img_class_mahalanobis)

# Evaluate classification accuracy
teste = measure.rasterize(shp_teste['geometry'], out_shape=raster_base.shape, fill=np.nan, transform=shp_teste.crs)
mask = ~np.isnan(teste)
masked_img_class_mahalanobis = img_class_mahalanobis.copy()
masked_img_class_mahalanobis[~mask] = np.nan

pix_teste_mahalanobis = masked_img_class_mahalanobis.ravel()[mask.ravel()]
pix_teste = teste.ravel()[mask.ravel()]

# Remove NaN values
non_nan_mask = ~np.isnan(pix_teste_mahalanobis)
pix_teste_mahalanobis = pix_teste_mahalanobis[non_nan_mask]
pix_teste = pix_teste[non_nan_mask]

# Evaluate accuracy using confusion matrix
confusion_matrix_mahalanobis = confusion_matrix(pix_teste, pix_teste_mahalanobis)
print(confusion_matrix_mahalanobis)

# SVM classification
# Select 10% of each class
n_per_class = np.round(img['CLASSE'].value_counts() * 0.1).astype(int)

# Filter samples
filter_table = pd.DataFrame()
for class_label in class_labels:
    filter_lines = img.loc[img['CLASSE'] == class_label].sample(n=n_per_class[class_label])
    filter_table = pd.concat([filter_table, filter_lines])

# SVM training
svm_params = {'C': np.logspace(-5, 5, 11), 'gamma': np.logspace(-5, 5, 11)}
svm_tune = GridSearchCV(SVC(kernel='rbf'), param_grid=svm_params)
svm_tune.fit(filter_table[img_channel_names], filter_table['CLASSE'])

# SVM prediction
svm_mod = SVC(kernel='rbf', C=svm_tune.best_params_['C'], gamma=svm_tune.best_params_['gamma'])
svm_mod.fit(img[img_channel_names], img['CLASSE'])
svm_apply = svm_mod.predict(img[img_channel_names])

# Assign SVM class predictions to image
dataframe_new = img.copy()
dataframe_new['classe_predict'] = svm_apply
img_class_svm = dataframe_new['classe_predict'].values.reshape(raster_base.shape)

# Plot the classified image using scikit-image
io.imshow(img_class_svm)

# Evaluate classification accuracy
masked_img_class_svm = img_class_svm.copy()
masked_img_class_svm[~mask] = np.nan

pix_teste_svm = masked_img_class_svm.ravel()[mask.ravel()]

# Remove NaN values
pix_teste_svm = pix_teste_svm[non_nan_mask]

# Evaluate accuracy using confusion matrix
confusion_matrix_svm = confusion_matrix(pix_teste, pix_teste_svm)
print(confusion_matrix_svm)
